In [6]:
!pip install -q ultralytics supervision opencv-python tqdm


In [7]:
import cv2
import json
import os
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
import supervision as sv
from collections import defaultdict


In [8]:
import os

VIDEO_PATH = "/content/2025_12_15_15_24_16_4_dQCiGf.MP4"

OUTPUT_DIR = "/content/outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUTPUT_VIDEO = f"{OUTPUT_DIR}/annotated_output.mp4"
OUTPUT_JSON = f"{OUTPUT_DIR}/output.json"

print("Video exists:", os.path.exists(VIDEO_PATH))
print("Annotated video exists:", os.path.exists(OUTPUT_VIDEO))
print("JSON exists:", os.path.exists(OUTPUT_JSON))


Video exists: True
Annotated video exists: True
JSON exists: True


In [9]:
model = YOLO("yolov8n.pt")


In [10]:
tracker = sv.ByteTrack()

box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_thickness=1)


In [11]:
cap = cv2.VideoCapture(VIDEO_PATH)

original_fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print("FPS:", original_fps)
print("Frames:", total_frames)

# Create video writer (CORRECT PATH)
out = cv2.VideoWriter(
    OUTPUT_VIDEO,
    cv2.VideoWriter_fourcc(*"mp4v"),
    original_fps,
    (width, height)
)

print("VideoWriter opened:", out.isOpened())


FPS: 19.89601605650955
Frames: 5545
VideoWriter opened: True


In [12]:
def weight_proxy(bbox):
    x1, y1, x2, y2 = bbox
    return (x2 - x1) * (y2 - y1)


In [13]:
frame_idx = 0
counts_over_time = []
tracks_summary = {}
weight_estimates = defaultdict(list)
unique_ids_seen = set()

progress = tqdm(total=total_frames)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1
    progress.update(1)

    results = model(frame, conf=0.3, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    current_count = 0
    labels = []

    for i, bbox in enumerate(detections.xyxy):
        track_id = int(detections.tracker_id[i])
        current_count += 1
        unique_ids_seen.add(track_id)

        weight_estimates[track_id].append(weight_proxy(bbox))

        if track_id not in tracks_summary:
            tracks_summary[track_id] = {
                "track_id": track_id,
                "first_seen_frame": frame_idx
            }

        labels.append(f"ID:{track_id}")

    timestamp = frame_idx / original_fps
    counts_over_time.append({
        "timestamp": round(timestamp, 2),
        "count": current_count
    })

    frame = box_annotator.annotate(frame, detections)
    frame = label_annotator.annotate(frame, detections, labels)

    out.write(frame)

progress.close()
cap.release()
out.release()


100%|██████████| 5545/5545 [17:45<00:00,  5.21it/s]


In [16]:
#to check the above cell is perfectly running or not
import os
print(os.path.exists(OUTPUT_VIDEO))
print(os.path.getsize(OUTPUT_VIDEO))


True
1230423361


In [17]:
final_weight_estimates = {
    str(k): {
        "mean_weight_proxy": float(np.mean(v)),
        "unit": "relative_index"
    }
    for k, v in weight_estimates.items()
}

final_output = {
    "counts": counts_over_time,
    "unique_birds_counted": len(unique_ids_seen),
    "weight_estimates": final_weight_estimates,
    "artifacts": {
        "input_video": VIDEO_PATH,
        "annotated_video": OUTPUT_VIDEO
    }
}

with open(OUTPUT_JSON, "w") as f:
    json.dump(final_output, f, indent=2)

print("JSON saved:", OUTPUT_JSON)


JSON saved: /content/outputs/output.json


In [18]:
print("Annotated video exists:", os.path.exists(OUTPUT_VIDEO))
print("JSON exists:", os.path.exists(OUTPUT_JSON))

!ls -lh /content/outputs


Annotated video exists: True
JSON exists: True
total 1.2G
-rw-r--r-- 1 root root 1.2G Dec 23 11:24 annotated_output.mp4
-rw-r--r-- 1 root root 315K Dec 23 11:24 output.json


In [ ]:
#from IPython.display import Video
#Video(OUTPUT_VIDEO, embed=True)


In [1]:
from google.colab import files
import os

OUTPUT_VIDEO = "/content/outputs/annotated_output.mp4"
OUTPUT_JSON = "/content/outputs/output.json"

print("Annotated video exists:", os.path.exists(OUTPUT_VIDEO))
print("JSON exists:", os.path.exists(OUTPUT_JSON))

files.download(OUTPUT_JSON)

if os.path.exists(OUTPUT_VIDEO):
    files.download(OUTPUT_VIDEO)
else:
    print("Annotated video not found — skipping download.")


Annotated video exists: True
JSON exists: True


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
from google.colab import files
files.download("/content/outputs/annotated_output.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>